In [1]:
import numpy as np

import matplotlib.pyplot as plt
import matplotlib.colors as colors

import scipy.constants as c
from scipy import stats

from photutils.aperture import CircularAperture
from photutils.detection import DAOStarFinder
from photutils.aperture import aperture_photometry
from photutils.aperture import CircularAnnulus

from astropy.utils.data import get_pkg_data_filename
from astropy.wcs import WCS
from astropy import wcs
from astropy.io import fits
from astropy.stats import sigma_clipped_stats
from astropy import units as u

import os
import sys
from astropy.table import Table, join, vstack
from astropy.nddata.utils import extract_array
import pyregion
import tqdm 
import argparse

import integratedflux as flux
from importlib import reload
reload(flux)
import warnings
from astropy.utils.exceptions import AstropyWarning

In [38]:
directory_univ_I = r'/net/vdesk/data2/GoesaertW/Meerkat_Data/Abell_85/I_slices/'
# directory_univ_I + 
I_dir = r'/data2/goesaert/Abell_85_I_plane_freq1.fits'

region_name = "Abell85_catalog_BDSF_output_rmsbox.reg"

srl = fits.open(get_pkg_data_filename('Abell85_catalog_BDSF_rsl.fits'))[1].data

In [6]:
warnings.filterwarnings('ignore', category=AstropyWarning, append=True)
mask = flux.mask_regions(I_dir, region_name,'/data2/goesaert/testmask.fits', maskoutside=False)

Writing mask file to /data2/goesaert/testmask.fits


In [41]:
measurement = flux.integratedflux(I_dir, mask, srl, 0)

#uncertainty = flux.uncertainty_flux(I_dir, measurement, beamnumb, rms=0.01*measurement, delta_cal=0.1)
#print(measurement, beamnumb, uncertainty)

21


In [ ]:
np.nansum(np.sqrt(np.array([1,2,np.NaN])))